In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model, load_model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
IMAGE_SIZE = [224, 224]
train_path = '/content/drive/MyDrive/chest_xray/train'
valid_path = '/content/drive/MyDrive/chest_xray/test'

In [ ]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
for layer in vgg.layers:
    layer.trainable = False

In [ ]:
folders = glob('/content/drive/MyDrive/chest_xray/train/*')

In [ ]:
x = Flatten()(vgg.output)

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)
 
# creating a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
# Use the Image Data Generator to import the images from the dataset
from keras.preprocessing.image import ImageDataGenerator
 
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
 
test_datagen = ImageDataGenerator(rescale = 1./255)
 
# providing the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/chest_xray/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
 

 
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/chest_xray/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 5216 images belonging to 2 classes.
Found 5216 images belonging to 2 classes.
Found 5216 images belonging to 2 classes.
Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
# fiting the model
 
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/20
163/163 [==============================] - 3113s 19s/step - loss: 0.2207 - accuracy: 0.9116 - val_loss: 0.2322 - val_accuracy: 0.9119
Epoch 2/20
163/163 [==============================] - 3120s 19s/step - loss: 0.1031 - accuracy: 0.9615 - val_loss: 0.2332 - val_accuracy: 0.9151
Epoch 3/20
163/163 [==============================] - 3062s 19s/step - loss: 0.0958 - accuracy: 0.9653 - val_loss: 0.3426 - val_accuracy: 0.8942
Epoch 4/20
163/163 [==============================] - 3057s 19s/step - loss: 0.0841 - accuracy: 0.9672 - val_loss: 0.4282 - val_accuracy: 0.8814
Epoch 5/20
163/163 [==============================] - 3055s 19s/step - loss: 0.0745 - accuracy: 0.9701 - val_loss: 0.3480 - val_accuracy: 0.9022
Epoch 6/20
163/163 [==============================] - 3053s 19s/step - loss: 0.0705 - accuracy: 0.9747 - val_loss: 0.4851 - val_accuracy: 0.8830
Epoch 7/20
163/163 [==============================] - 3055s 19s/step - loss: 0.0742 - accuracy: 0.9758 - val_loss: 0.3776 - val_ac

In [ ]:
import tensorflow as tf
from keras.models import load_model
model.save('vgg16pneumonia.h5')